# MetaTrain
Trains the lstm optimizer model

### Imports and Setup

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from models import simple_training_NN
from models import l2o_lstm_model

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
# Hyperparameters

batch_size = 128
epochs = 10

In [5]:
# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

print(f"Number of batches: {len(train_dataloader)}")

Number of batches: 469


In [6]:
optimizer = l2o_lstm_model()

### Meta-Training Process

In [7]:
optimizee = simple_training_NN()
loss_fn = nn.CrossEntropyLoss()

In [8]:
X, y = iter(train_dataloader).next()

In [9]:
pred = optimizee(X)
loss = loss_fn(pred, y)

# backprop
loss.backward()

In [10]:
for p in optimizee.parameters():
    print(p.grad.shape)

torch.Size([32, 784])
torch.Size([32])
torch.Size([10, 32])
torch.Size([10])


In [11]:
for p in optimizee.parameters():
    # Input shape to LSTM: (seq_len, batch_size, input_dim)
    update = optimizer(p.grad.reshape(1, -1, 1))

    update = update.reshape(p.grad.shape)

    new_val = p - update

    with torch.no_grad():
        p.copy_(new_val)


RuntimeError: shape '[32, 784]' is invalid for input of size 1

In [17]:
p

Parameter containing:
tensor([ 0.0062,  0.1132, -0.0846,  0.1562, -0.1198, -0.1100, -0.0647,  0.1304,
         0.0914, -0.0705], requires_grad=True)